In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import time


from sklearn.neural_network import MLPClassifier

from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import classification_report,confusion_matrix

#df = pd.read_csv("C:\\Users\Eric\Desktop\crash_data_one_hot_encoded_MLP.csv")
df = pd.read_csv("crash_data_no_unknowns_clustered.csv")
df2 = pd.read_csv("crash_data_only_numeric_values.csv")

#add crash_sev_code to one_hot_encoded
df = df.assign(CRASH_SEV_CODE = df2["CRASH_SEV_CODE"])
df.drop('CRASH_SEV_F', axis='columns', inplace=True)
df.drop('CRASH_SEV_M', axis='columns', inplace=True)
df.drop('CRASH_SEV_N', axis='columns', inplace=True)
df.drop('CRASH_SEV_S', axis='columns', inplace=True)
#for cluster_scaled
df.drop('CLUSTER', axis='columns', inplace=True)
#for cluster
#df.drop('CLUSTER_SCALED', axis='columns', inplace=True)

df = sklearn.utils.shuffle(df)
df.fillna(0,inplace=True)
encode = LabelEncoder()

#features = ['CRASH_YEAR', 'NUM_LANES', 'SPD_LIM', 'MULTI_VEH_CODE','HOLIDAY_CODE',
#            'LG_REGION_DESC_CODE', 'JUNCTION_TYPE_CODE', 'DIRN_ROLE1_DESC_CODE','INTSN_MIDBLOCK_CODE',
#            'FLAT_HILL_CODE','ROAD_CURVATURE_CODE','ROAD_MARKINGS_CODE','ROAD_SURFACE_CODE','ROAD_WET_CODE',
#            'URBAN_CODE','LIGHT_CODE','STREET_LIGHT_CODE','WEATHER_A_CODE']

features = ['CRASH_YEAR', 
'NUM_LANES', 
'SPD_LIM', 
'TRAFFIC_CTRL_Give Way Sign', 
'TRAFFIC_CTRL_Nil', 
'TRAFFIC_CTRL_Points Man', 
'TRAFFIC_CTRL_School Patrol', 
'TRAFFIC_CTRL_Stop Sign', 
'TRAFFIC_CTRL_Traffic Signal', 
'MULTI_VEH_Cyclist(s)+Pedestrian(s) only', 
'MULTI_VEH_Cyclists only', 
'MULTI_VEH_Multi vehicle', 
'MULTI_VEH_Other', 
'MULTI_VEH_Others without non-parked veh', 
'MULTI_VEH_Single vehicle', 
'MULTI_VEH_Vehicle(s)+Cyclist(s) only', 
'MULTI_VEH_Vehicle(s)+Pedestrian(s)', 
'MULTI_VEH_Vehicle(s)+multiple other types', 
'HOLIDAY_Christmas/New Year', 
'HOLIDAY_Easter', 
'HOLIDAY_Labour Weekend', 
'HOLIDAY_None', 
'HOLIDAY_Queens Birthday', 
'LG_REGION_DESC_0', 
'LG_REGION_DESC_Auckland            ', 
'LG_REGION_DESC_Bay of Plenty       ', 
'LG_REGION_DESC_Canterbury          ', 
'LG_REGION_DESC_Gisborne            ', 
'LG_REGION_DESC_Hawkes Bay          ', 
'LG_REGION_DESC_Manawatu/Wanganui   ', 
'LG_REGION_DESC_Nelson/Marlborough  ', 
'LG_REGION_DESC_Northland           ', 
'LG_REGION_DESC_Otago               ', 
'LG_REGION_DESC_Southland           ', 
'LG_REGION_DESC_Taranaki            ', 
'LG_REGION_DESC_Waikato             ', 
'LG_REGION_DESC_Wellington          ', 
'LG_REGION_DESC_West Coast          ', 
'JUNCTION_TYPE_Driveway', 
'JUNCTION_TYPE_Multi Rd Join', 
'JUNCTION_TYPE_Roundabout', 
'JUNCTION_TYPE_T Type Junction',  
'JUNCTION_TYPE_X Type Junction', 
'JUNCTION_TYPE_Y Type Junction', 
'DIRN_ROLE1_DESC_0', 
'DIRN_ROLE1_DESC_East', 
'DIRN_ROLE1_DESC_North', 
'DIRN_ROLE1_DESC_South',  
'DIRN_ROLE1_DESC_West', 
'INTSN_MIDBLOCK_Intersection', 
'INTSN_MIDBLOCK_Mid Block', 
'FLAT_HILL_Flat', 
'FLAT_HILL_Hill',  
'ROAD_CURVATURE_Easy Curve', 
'ROAD_CURVATURE_Moderate Curve', 
'ROAD_CURVATURE_Severe Curve', 
'ROAD_CURVATURE_Straight Road',  
'ROAD_MARKINGS_Centre Line', 
'ROAD_MARKINGS_No Marks', 
'ROAD_MARKINGS_No Passing Lines', 
'ROAD_MARKINGS_Painted Island', 
'ROAD_MARKINGS_Ped Crossing', 
'ROAD_MARKINGS_Raised Island', 
'ROAD_SURFACE_Sealed', 
'ROAD_SURFACE_Unsealed', 
'ROAD_WET_Dry', 
'ROAD_WET_Ice/ Snow', 
'ROAD_WET_Wet', 
'URBAN_Openroad', 
'URBAN_Urban', 
'LIGHT_Bright Sun', 
'LIGHT_Dark', 
'LIGHT_Overcast', 
'LIGHT_Twilight',  
'STREET_LIGHT_None', 
'STREET_LIGHT_Off', 
'STREET_LIGHT_On',  
'WEATHER_A_Fine', 
'WEATHER_A_Heavy Rain', 
'WEATHER_A_Light Rain', 
'WEATHER_A_Mist', 
'WEATHER_A_Snow',
'CLUSTER_SCALED']

df_non_injury = df[df.CRASH_SEV_CODE==2]  # 480452 samples
df_minor = df[df.CRASH_SEV_CODE==1]       # 150834 samples
df_serious = df[df.CRASH_SEV_CODE==3]    # 37347  samples
df_fatal = df[df.CRASH_SEV_CODE==0]       # 6178   samples

df_minor_upsampled = resample(df_minor, replace=True,n_samples=480452,random_state=40)
df_serious_upsampled = resample(df_serious, replace=True,n_samples=480452,random_state=40)
df_fatal_upsampled = resample(df_fatal, replace=True,n_samples=480452,random_state=40)

df_upsampled = pd.concat([df_non_injury, df_minor_upsampled, df_serious_upsampled, df_fatal_upsampled])

df_upsampled.CRASH_SEV_CODE.value_counts()




3    480452
2    480452
1    480452
0    480452
Name: CRASH_SEV_CODE, dtype: int64

In [8]:
X = df_upsampled.drop(["CRASH_SEV_CODE"], axis=1).values
y = df_upsampled["CRASH_SEV_CODE"].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=40)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
print(X_train.shape); print(X_test.shape)

(1729627, 83)
(192181, 83)


In [9]:
mlp = MLPClassifier(hidden_layer_sizes=(84,42,1), activation='relu', solver='adam', max_iter=500,verbose = True)

start = time.time()
mlp.fit(X_train,y_train)
stop = time.time()

predict_train = mlp.predict(X_train)
predict_test = mlp.predict(X_test)
print(f"\n Training time: {stop - start}s")
print("\n training data\n")
print(confusion_matrix(y_train,predict_train))
print("\n")
print(classification_report(y_train,predict_train,labels=pd.unique(predict_train)))
print("\n---------------------------------------------------------------------------------------\n")
print("\n test data\n")
print(confusion_matrix(y_test,predict_test))
print("\n")
print(classification_report(y_test,predict_test,labels=pd.unique(predict_test)))

Iteration 1, loss = 1.16832581
Iteration 2, loss = 1.07541282
Iteration 3, loss = 1.03991732
Iteration 4, loss = 1.02180869
Iteration 5, loss = 1.01160595
Iteration 6, loss = 1.00418975
Iteration 7, loss = 0.99830784
Iteration 8, loss = 0.99282192
Iteration 9, loss = 0.98880303
Iteration 10, loss = 0.98537841
Iteration 11, loss = 0.98202037
Iteration 12, loss = 0.97958597
Iteration 13, loss = 0.97715531
Iteration 14, loss = 0.97484923
Iteration 15, loss = 0.97286178
Iteration 16, loss = 0.97071261
Iteration 17, loss = 0.96925404
Iteration 18, loss = 0.96761022
Iteration 19, loss = 0.96593055
Iteration 20, loss = 0.96468667
Iteration 21, loss = 0.96316838
Iteration 22, loss = 0.96212220
Iteration 23, loss = 0.96070175
Iteration 24, loss = 0.95944361
Iteration 25, loss = 0.95865197
Iteration 26, loss = 0.95740534
Iteration 27, loss = 0.95672822
Iteration 28, loss = 0.95582245
Iteration 29, loss = 0.95498976
Iteration 30, loss = 0.95402917
Iteration 31, loss = 0.95320513
Iteration 32, los

              precision    recall  f1-score   support

           0       0.81      0.99      0.89     48052
           3       0.45      0.65      0.53     48324
           2       0.57      0.45      0.50     47820
           1       0.36      0.18      0.24     47985

    accuracy                           0.57    192181
   macro avg       0.54      0.57      0.54    192181
weighted avg       0.54      0.57      0.54    192181



In [35]:
classification_report(y_test,predict_test,labels=pd.unique(predict_test))
print(mlp.classes_)

[0 1 2 3]


In [17]:
# only run this cell and below for feature importance
from sklearn.linear_model import Perceptron
clf = Perceptron(tol=1e-3, random_state=40, verbose=True,n_jobs = -1)
clf.fit(X,y)

coeffs = clf.coef_

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.


-- Epoch 1-- Epoch 1

-- Epoch 1-- Epoch 1

Norm: 195158.69, NNZs: 18, Bias: 23.000000, T: 1921808, Avg. loss: 686148.332722
Total training time: 0.49 seconds.
-- Epoch 2
Norm: 82456.47, NNZs: 18, Bias: -59.000000, T: 1921808, Avg. loss: 745568.259439
Total training time: 0.49 seconds.
-- Epoch 2
Norm: 116749.24, NNZs: 18, Bias: -29.000000, T: 1921808, Avg. loss: 744053.898060
Total training time: 0.52 seconds.
-- Epoch 2
Norm: 78961.63, NNZs: 18, Bias: 11.000000, T: 1921808, Avg. loss: 672685.401437
Total training time: 0.52 seconds.
-- Epoch 2
Norm: 281231.62, NNZs: 18, Bias: 43.000000, T: 3843616, Avg. loss: 668819.152235
Total training time: 0.98 seconds.
-- Epoch 3
Norm: 128339.92, NNZs: 18, Bias: -118.000000, T: 3843616, Avg. loss: 743692.985429
Total training time: 1.01 seconds.
-- Epoch 3
Norm: 164033.92, NNZs: 18, Bias: -53.000000, T: 3843616, Avg. loss: 736939.681098
Total training time: 1.02 seconds.
-- Epoch 3
Norm: 94703.29, NNZs: 18, Bias: 21.000000, T: 3843616, Avg. loss

[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:   10.6s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:   10.6s finished


In [40]:
features = df.drop(["CRASH_SEV_CODE"], axis=1).columns
minV = 0
maxV = 0
minPlace = 0
maxPlace = 0
count = 0
for i in coeffs:
    for j in i:
        if count == 0:
            minPlace = 0
            maxPlace = 0
            minV = j
            maxV = j
        else:
            if minV > j:
                minV =j
                minPlace = count
            if maxV < j:
                maxV = j
                maxPlace = count
        count+=1
    
    count = 0
    print("-------------------")
    print(" min: ",minV," feature: ",features[minPlace],"\n max: ",maxV," feature: ",features[maxPlace])
    print("-------------------")

predict_train2 = clf.predict(X_train)
predict_test2 = clf.predict(X_test)

print("\n training data\n")
print(confusion_matrix(y_train,predict_train2))
print("\n")
print(classification_report(y_train,predict_train2,labels=pd.unique(predict_train)))
print("\n---------------------------------------------------------------------------------------\n")
print("\n test data\n")
print(confusion_matrix(y_test,predict_test2))
print("\n")
print(classification_report(y_test,predict_test2,labels=pd.unique(predict_train)))

-------------------
 min:  -657.505707130214  feature:  ROAD_SURFACE_Unknown 
 max:  37.03248869219892  feature:  MULTI_VEH_Other
-------------------
-------------------
 min:  -1315.1133036655588  feature:  DIRN_ROLE1_DESC_0 
 max:  657.6182417089478  feature:  ROAD_SURFACE_Unknown
-------------------
-------------------
 min:  -657.5057071302133  feature:  ROAD_SURFACE_Unknown 
 max:  1315.2258382442915  feature:  DIRN_ROLE1_DESC_0
-------------------
-------------------
 min:  -657.5193937681679  feature:  ROAD_SURFACE_Unknown 
 max:  41.12664954461998  feature:  MULTI_VEH_Cyclist(s)+Pedestrian(s) only
-------------------

 training data

[[ 94361 101434  51191 185414]
 [102238 125102 112274  92853]
 [ 82660 123299 161536  65137]
 [ 98839 110351  85673 137265]]


              precision    recall  f1-score   support

           0       0.25      0.22      0.23    432400
           2       0.39      0.37      0.38    432632
           1       0.27      0.29      0.28    432467
      

numpy.ndarray